IDEA: allenare un modello validatore di taglia piccola per aiutare tutti i modelli tramite agente, per ottenere risultati migliori nel parsing

# TEST VALIDATORE

### Impostazione 
- Le difficoltà nelle rilevazione da parte del validatore sono passate nella costruzione del system prompt the validatore, per farci aiutare a rilevare gli errori, che non si riescono a risolvere in modo automatico.

### Difficoltà riscontrate nella rilveazione degli errori:

#### 1 Detecting the nested parameters are part of a list, from the function definition. example
- Errore risposta:  `array_sort(list=[5, 3, 4, 1, 2], order=ascending)`
- Possibile risposta: `{'array_sort': {'list': [[5.0, 3.0, 4.0, 1.0, 2.0]], 'order': ['ascending']}}]`
- Validatore attuale: `{'valid': True, 'error': [], 'error_type': 'simple_function_checker:unclear'}`
- BFCL validatore: `["Nested type checking failed for parameter 'list'. Expected outer type array with inner type <class 'float'>. Parameter value: [5, 3, 4, 1, 2]."]`

Soluzione: integrare la gestione del validatore facendo riferimento alla descrizione della funzione al posto della `possible answer`, o istruire un modello a fare attenzione ai tipi `List`.
Indicazione: if you find the `Nested type checking failed for parameter` check the nested parameters are part of a list, from the function definition.

#### 2 Detecting the missing of "" example
- Errore risposta: `crime_record.get_record(case_number=CA123456, county=San Diego County, details=True)`
- Possibile risposta: `[{'crime_record.get_record': {'case_number': ['CA123456'], 'county': ['San Diego', 'San Diego County'], 'details': [True]}}]`
- Validatore attuale: `['Invalid syntax. Failed to decode AST. invalid syntax. Perhaps you forgot a comma? (<unknown>, line 1)']`
- BFCL validatore: `['Invalid syntax. Failed to decode AST. invalid syntax. Perhaps you forgot a comma? (<unknown>, line 1)'] `

Soluzione: Indicare al modello come trasformare errore del validatore in un azione per correggere l'errore.
Indicazione: if you find the `Perhaps you forgot a comma?` check the missing of "" in the parameters and put it to assure the correct syntax.

#### 3 Detecting the correct value example
- Errore risposta: `get_news(topic=BTC, quantity=5, region='US')`
- Possibile risposta: `[{'get_news': {'topic': ['Bitcoin'], 'quantity': [5], 'region': ['US', '']}}]`
- Validatore attuale: `{'valid': True, 'error': [], 'error_type': 'simple_function_checker:unclear'} `
- BFCL validatore: `["Invalid value for parameter 'topic': 'BTC'. Expected one of ['Bitcoin']. Case insensitive."]`

Soluzione: Avere la rispsta, ma non sarebbe lecito perché il ground truth non è accessibile, quindi è il motivo per cui è stato costruito il validatore interno. Che per evitare confusioni, lo chiameremo compilatore (del function calling). Come analogia sarebbe il compilatore del codice per i linguaggi di programmazione. (per poi fare un confronto con il paper Agent-Coder)

#### 4 Detecting missing ()

- Errore risposta: `history_eu_fetch_events_events_century=19th_century, region=Western, category=Wars)`
- Possibile risposta:`[{'history_eu.fetch_events': {'century': [19], 'region': ['Northern', 'Southern', 'Eastern', 'Western'], 'category': ['Wars']}}]`
- Validatore attuale: `['Invalid syntax. Failed to decode AST. invalid decimal literal (<unknown>, line 1)']`
- BFCL validatore: `['Invalid syntax. Failed to decode AST. invalid decimal literal (<unknown>, line 1)']`  

Soluzione: tabulare tutte le casistiche di invalid decimal literal per poterli gestire sepratamente le casistiche che capitano. Per tabulare posso guardare intanto tutte le `invalid decimal literal` e tra questi le tipologie di errore che si presentano ed elencarle                                                    
Indicazione: The parameters of the function must be enclosed in parentheses. Check the missing () in the function call.

#### 5

- Errore risposta: 
- Possibile risposta:
- Validatore attuale:
- BFCL validatore:                                                                   


## Errori inviduati sul valutatore di BFCL

### 1 Errore
In questo caso viene richiesto esplicitamente segnalato come errore la mancanza di un parametro opzionale: `root_type`. Quando non viene espresso nella lista dei valori richiesti per la corretta esecuzione della funzione. Il modello quindi è indotto ad una risposta sbagliata basata su una gestione backend (quella di inserire possibili funzioni da chiamare)


Il testo dell'errore considerato dal BFCL evaluate è il seguente:

{"id": 6, "model_name": "Qwen_Qwen2.5-3B-Instruct", "test_category": "simple", "valid": false, "error": ["Optional parameter 'root_type' not provided and not marked as optional."], "error_type": "simple_function_checker:missing_optional", "prompt": {"id": "simple_5", "question": [[{"role": "user", "content": "Find all the roots of a quadratic equation given coefficients a = 3, b = -11, and c = -4."}]], "function": [{"name": "solve_quadratic", "description": "Solve a quadratic equation given coefficients a, b, and c. If optional 'root_type' is 'real', the function will only return real roots. If not specified, function may return complex roots.", "parameters": {"type": "dict", "properties": {"a": {"type": "integer", "description": "The coefficient of the squared term in the quadratic equation."}, "b": {"type": "integer", "description": "The coefficient of the linear term in the quadratic equation."}, "c": {"type": "integer", "description": "The constant term in the quadratic equation."}, "root_type": {"type": "string", "description": "The type of roots to return: 'real' for real roots, 'all' for both real and complex roots. Default value is 'real'."}}, "required": ["a", "b", "c"]}}]}, "model_result_raw": "[solve_quadratic(a=3, b=-11, c=-4)]", "model_result_decoded": [{"solve_quadratic": {"a": 3, "b": -11, "c": -4}}], "possible_answer": [{"solve_quadratic": {"a": [3], "b": [-11], "c": [-4], "root_type": ["all"]}}]}

Di seguito l'elenco di quelli avvistati:

{"id": 68, "model_name": "Qwen_Qwen2.5-3B-Instruct", "test_category": "simple", "valid": false, "error": ["Optional parameter 'size' not provided and not marked as optional."], "error_type": "simple_function_checker:missing_optional", "prompt": {"id": "simple_67", "question": [[{"role": "user", "content": "Identify a small green bird in forest."}]], "function": [{"name": "identify_bird", "description": "Identify a bird species based on certain characteristics.", "parameters": {"type": "dict", "properties": {"color": {"type": "string", "description": "Color of the bird."}, "habitat": {"type": "string", "description": "Habitat of the bird."}, "size": {"type": "string", "enum": ["small", "medium", "large"], "description": "Size of the bird. Default is 'small'"}}, "required": ["color", "habitat"]}}]}, "model_result_raw": "identify_bird(color=\"green\", habitat=\"forest\")", "model_result_decoded": [{"identify_bird": {"color": "green", "habitat": "forest"}}], "possible_answer": [{"identify_bird": {"color": ["green"], "habitat": ["forest"], "size": ["small"]}}]}


{"id": 70, "model_name": "Qwen_Qwen2.5-3B-Instruct", "test_category": "simple", "valid": false, "error": ["Optional parameter 'species' not provided and not marked as optional."], "error_type": "simple_function_checker:missing_optional", "prompt": {"id": "simple_69", "question": [[{"role": "user", "content": "Find out the population and species of turtles in Mississippi river in 2020."}]], "function": [{"name": "ecology.get_turtle_population", "description": "Get the population and species of turtles in a specific location.", "parameters": {"type": "dict", "properties": {"location": {"type": "string", "description": "The name of the location."}, "year": {"type": "integer", "description": "The year of the data requested. Default is 2001."}, "species": {"type": "boolean", "description": "Whether to include species information. Default is false."}}, "required": ["location"]}}]}, "model_result_raw": "ecology.get_turtle_population(location=\"Mississippi river\", year=2020)", "model_result_decoded": [{"ecology.get_turtle_population": {"location": "Mississippi river", "year": 2020}}], "possible_answer": [{"ecology.get_turtle_population": {"location": ["Mississippi river"], "year": [2020], "species": [true]}}]}

{"id": 272, "model_name": "Qwen_Qwen2.5-3B-Instruct", "test_category": "simple", "valid": false, "error": ["Optional parameter 'mode' not provided and not marked as optional."], "error_type": "simple_function_checker:missing_optional", "prompt": {"id": "simple_271", "question": [[{"role": "user", "content": "What is the structural dynamic analysis of the building with building Id B1004 for 2nd, 3rd and 4th floors?"}]], "function": [{"name": "analyze_structure", "description": "Analyze a structure of a building based on its Id and floor numbers.", "parameters": {"type": "dict", "properties": {"building_id": {"type": "string", "description": "The unique identification number of the building."}, "floors": {"type": "array", "items": {"type": "integer"}, "description": "Floor numbers to be analyzed."}, "mode": {"type": "string", "description": "Mode of analysis, e.g. 'static' or 'dynamic'. Default is 'static'."}}, "required": ["building_id", "floors"]}}]}, "model_result_raw": "analyze_structure(building_id=B1004, floors=[2, 3, 4])", "model_result_decoded": [{"analyze_structure": {"building_id": "B1004", "floors": [2, 3, 4]}}], "possible_answer": [{"analyze_structure": {"building_id": ["B1004"], "floors": [[2, 3, 4]], "mode": ["dynamic"]}}]}



#### DA CONTROLLARE 

send_email(to=John.Doe@example.com, subject='Meeting', body='Let's meet at 10 AM tomorrow')                                      | [{'send_email': {'to': ['john.doe@example.com'], 'subject': ['Meeting'], 'body': ["Let's meet at 10 AM tomorrow", "Let's meet at 10 AM tomorrow."], 'cc': [''], 'bcc': ['']}}] | ['Invalid syntax. Failed to decode AST. unterminated string literal (detected at line 1) (<unknown>, line 1)']                                                            | {'valid': False, 'error': ['Invalid syntax. Failed to decode AST. unterminated string literal (detected at line 1) (<unknown>, line 1)'], 'error_type': 'ast_decoder:decoder_failed'}                                                   |
+----------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| sentiment_analysis(text='I love the food here! It's always fresh and delicious.', language='en')                                 | [{'sentiment_analysis': {'text': ["I love the food here! It's always fresh and delicious."], 'language': ['english', 'English', 'en']}}]                                       | ['Invalid syntax. Failed to decode AST. unterminated string literal (detected at line 1) (<unknown>, line 1)']                                                            | {'valid': False, 'error': ['Invalid syntax. Failed to decode AST. unterminated string literal (detected at line 1) (<unknown>, line 1)'], 'error_type': 'ast_decoder:decoder_failed'}   


| boardgame.get_info(name="Ticket to Ride", parameters="rating", player_count)                                                                                                                                                      | [{'boardgame.get_info': {'name': ['Ticket to Ride'], 'parameters': [['rating', 'player count'], ['player count', 'rating']], 'language': ['', 'English']}}]                                                                                                                                       | ['Invalid syntax. Failed to decode AST. positional argument follows keyword argument (<unknown>, line 1)']                                 | {'valid': False, 'error': ['Invalid syntax. Failed to decode AST. positional argument follows keyword argument (<unknown>, line 1)'], 'error_type': 'ast_decoder:decoder_failed'}  





In [ ]:
from validator_utils import is_java, is_js, estrai_e_stampa_dati_con_validator,ast_checker,validate_and_print, estrai_e_conta_validita_dataset, analisi_score_completo
# Percorso al file JSON
# file_path = '/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-3B-Instruct/BFCL_v3_multiple_score.json'

# DATI CONTROLLATI:
# - Simple | Qwen3B | BFCL_v3_simple_score.json

# Mostra i primi 5 elementi
# df = estrai_e_stampa_dati_con_validator(file_path, start=1, end=80,language='Python')
#risultato = estrai_e_conta_validita_dataset(file_path,language='Python')


# files_path = [('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-3B-Instruct/BFCL_v3_simple_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-3B-Instruct/BFCL_v3_multiple_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-3B-Instruct/BFCL_v3_parallel_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-3B-Instruct/BFCL_v3_parallel_multiple_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-3B-Instruct/BFCL_v3_java_score.json', 'Java'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-3B-Instruct/BFCL_v3_javascript_score.json', 'JavaScript')
#               ]

# files_path = [('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_simple_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_multiple_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_parallel_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_parallel_multiple_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_java_score.json', 'Java'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_javascript_score.json', 'JavaScript')
#               ]

# files_path = [('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/meta-llama_Llama-3.1-8B-Instruct/BFCL_v3_simple_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/meta-llama_Llama-3.1-8B-Instruct/BFCL_v3_multiple_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/meta-llama_Llama-3.1-8B-Instruct/BFCL_v3_parallel_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/meta-llama_Llama-3.1-8B-Instruct/BFCL_v3_parallel_multiple_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/meta-llama_Llama-3.1-8B-Instruct/BFCL_v3_java_score.json', 'Java'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/meta-llama_Llama-3.1-8B-Instruct/BFCL_v3_javascript_score.json', 'JavaScript')
#               ]
files_path = [('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/meta-llama_Llama-3.2-3B-Instruct/BFCL_v3_simple_score.json', 'Python'),
              ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/meta-llama_Llama-3.2-3B-Instruct/BFCL_v3_multiple_score.json', 'Python'),
              ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/meta-llama_Llama-3.2-3B-Instruct/BFCL_v3_parallel_score.json', 'Python'),
              ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/meta-llama_Llama-3.2-3B-Instruct/BFCL_v3_parallel_multiple_score.json', 'Python'),
              ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/meta-llama_Llama-3.2-3B-Instruct/BFCL_v3_java_score.json', 'Java'),
              ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/meta-llama_Llama-3.2-3B-Instruct/BFCL_v3_javascript_score.json', 'JavaScript')
              ]
# files_path = [('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-1.5B-Instruct/BFCL_v3_simple_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-1.5B-Instruct/BFCL_v3_multiple_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-1.5B-Instruct/BFCL_v3_parallel_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-1.5B-Instruct/BFCL_v3_parallel_multiple_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-1.5B-Instruct/BFCL_v3_java_score.json', 'Java'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-1.5B-Instruct/BFCL_v3_javascript_score.json', 'JavaScript')
#               ]
analisi_score_completo = analisi_score_completo(files_path)


--- Totale delle Righe per File e Linguaggio ---
+---------------------------------+--------------+----------------+
| File                            | Linguaggio   |   Totale Righe |
+=================================+==============+================+
| BFCL_v3_simple_score            | Python       |             34 |
+---------------------------------+--------------+----------------+
| BFCL_v3_multiple_score          | Python       |             24 |
+---------------------------------+--------------+----------------+
| BFCL_v3_parallel_score          | Python       |             59 |
+---------------------------------+--------------+----------------+
| BFCL_v3_parallel_multiple_score | Python       |             71 |
+---------------------------------+--------------+----------------+
| BFCL_v3_java_score              | Java         |             40 |
+---------------------------------+--------------+----------------+
| BFCL_v3_javascript_score        | JavaScript   |             15 

In [ ]:
from validator_utils import is_java, is_js, estrai_e_stampa_dati_con_validator,ast_checker,validate_and_print, estrai_e_conta_validita_dataset, analisi_score_completo

files_path = [('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-agent-v003/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_simple_score.json', 'Python'),
              ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-agent-v003/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_multiple_score.json', 'Python'),
              ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-agent-v003/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_parallel_score.json', 'Python'),
              ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-agent-v003/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_parallel_multiple_score.json', 'Python'),
              ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-agent-v003/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_java_score.json', 'Java'),
              ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-agent-v003/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_javascript_score.json', 'JavaScript')
              ]

# files_path = [('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_simple_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_multiple_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_parallel_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_parallel_multiple_score.json', 'Python'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_java_score.json', 'Java'),
#               ('/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-7B-Instruct/BFCL_v3_javascript_score.json', 'JavaScript')
#               ]

analisi_score_completo = analisi_score_completo(files_path)



--- Totale delle Righe per File e Linguaggio ---
+---------------------------------+--------------+----------------+
| File                            | Linguaggio   |   Totale Righe |
+=================================+==============+================+
| BFCL_v3_simple_score            | Python       |             24 |
+---------------------------------+--------------+----------------+
| BFCL_v3_multiple_score          | Python       |             11 |
+---------------------------------+--------------+----------------+
| BFCL_v3_parallel_score          | Python       |             23 |
+---------------------------------+--------------+----------------+
| BFCL_v3_parallel_multiple_score | Python       |             36 |
+---------------------------------+--------------+----------------+
| BFCL_v3_java_score              | Java         |             36 |
+---------------------------------+--------------+----------------+
| BFCL_v3_javascript_score        | JavaScript   |             14 

In [ ]:
from validator_function import extract_and_analyze_single_function,estrai_model_result_raw_and_function
import json

# Percorso al file JSON
file_path = '/home/ago/giammy/TESI-MAGISTRALE/thesis/data/score-quantize/Qwen_Qwen2.5-3B-Instruct/BFCL_v3_simple_score.json'
total_model_result_raw_and_function = estrai_model_result_raw_and_function(file_path,start=2)

print(total_model_result_raw_and_function)

for idx, x in enumerate(total_model_result_raw_and_function, start=1):

    function_definitions = x['function']
    result = extract_and_analyze_single_function(
        input_str=x['model_result_raw'],
        predefined_functions=function_definitions,
        similarity_threshold=0.1,  # Adjust as needed
        parameter_similarity_threshold=0.8,
        case_sensitive=False
    )
    print(f"Test Case {idx}:")
    print(json.dumps(result, indent=4))
    print("-" * 50)